# Transformations and Interactions

[Resource](https://online.stat.psu.edu/stat462/node/85/)

Now it's time to take a closer look at transformations and interactions using an **absolutely amazing** resource provided by Penn State. Amazing how much free knowledge is out there. Might even be worth doing a full run through this entire resource later.

Unfortunately, this resource seems to be strictly for R programming applications, but no matter! If anything, translating these notes into code will only provide more practice for the upcoming lessons.

Let's get to it.

## MLR Model Assumptions - LINE

These points come from a previous section found [here](https://online.stat.psu.edu/stat462/node/145/).

* The mean of the response at each set of values of the predictors is a **Linear function** of the predictors.
* The errors are **Independent**.
* The errors at each set of values of the predictors are **Normally distributed**.
* The errors at each set of values of the predictors have **Equal variances**.

An equivalent way to think of the first condition is that the mean of the error at each set of values of the predictors is **zero**. An alternative way to describe all four assumptions is that the errors are independent normal random variables with mean zero and constant variance.

We can assess whether these conditions seem to hold for a MLR model applied to a particular sample dataset by looking at the estimated errors, i.e., the residuals (actual - predicted).

# Log-transforming Only the Predictor for SLR

In this section, we learn how to build and use a simple linear regression model by transforming the predictor *x* values. **Transforming the *x* values is appropriate when non-linearity is the only problem** (i.e., the independence, normality, and equal variance conditions are met). 